In [1]:
%pip install -U pymilvus sentence-transformers fsspec s3fs
%pip install --upgrade huggingface-hub

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.9.0 which is incompatible.
cloud-ml 0.0.1 requires requests<=2.28.1,>=2.22.0, but you have requests 2.32.3 which is incompatible.
cloud-ml 0.0.1 requires s3fs<=0.5.2,>=0.4.1, but you have s3fs 2024.9.0 which is incompatible.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2024.9.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.17.

In [2]:
import torch

print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

Is CUDA available: True
CUDA device: Tesla T4


In [3]:
filename = "npa_dataset_v4.db"  # change dataset version when new iteration begins!
database_path = f"/home/jupyter/datasphere/s3/hack-object-storage/database/{filename}"
dataset_path = (
    "/home/jupyter/datasphere/project/preprocessed_dataset_ver_4.txt"
)

In [4]:
with open(dataset_path, encoding="utf-8") as file:
    dataset_text = file.read()

data_raw = dataset_text.split("</s>\n")
data_raw[:5]

['постановление губернатора ханты-мансийского автономного округа-югры от 28.12.2017 № 139. о внесении изменений в приложение к постановлению губернатора ханты-мансийского автономного округа – югры от 30 декабря 2012 года n 176 "об инструкции по делопроизводству в государственных органах ханты-мансийского автономного округа - югры и исполнительных органах государственной власти ханты-мансийского автономного округа - югры" губернатор ханты-мансийского автономного округа - югры постановление от 28 декабря 2017 г. n 139 о внесении изменений в приложение к постановлению губернатора ханты-мансийского автономного округа – югры от 30 декабря 2012 года n 176 "об инструкции по делопроизводству в государственных органах ханты-мансийского автономного округа - югры и исполнительных органах государственной власти ханты-мансийского автономного округа - югры" руководствуясь законами ханты-мансийского автономного округа - югры от 19 ноября 2001 года n 75-оз "о губернаторе ханты-мансийского автономного 

In [5]:
# from sentence_transformers import SentenceTransformer
# from transformers import AutoModel

# embedding_dim = 1024

# model_sentence = SentenceTransformer("deepvk/USER-bge-m3")
# model_sentence = model_sentence.to("cuda")

In [6]:
# print("==== Embeddings generation in progress... ====")
# embeddings = model_sentence.encode(data_raw, normalize_embeddings=True, batch_size=16)
# print("Done.")

In [7]:
from langchain.docstore.document import Document

documents = []
for texts in data_raw:
    doc = Document(page_content=texts, metadata={"source": dataset_path, "database_path": database_path})
    documents.append(doc)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_milvus import Milvus
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel

model = AutoModel.from_pretrained("deepvk/USER-bge-m3", trust_remote_code=True) 

model_name = "deepvk/USER-bge-m3"
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

# Retriever
# API Here: https://python.langchain.com/v0.2/api_reference/milvus/vectorstores/langchain_milvus.vectorstores.milvus.Milvus.html
vectorstore = Milvus.from_documents(
    documents = documents,
    collection_name = "npa_storage_1024_128", 
    connection_args={"uri": database_path},
    embedding = embeddings
)

# retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-09-12 14:48:41.700387: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 14:48:43.371883: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 14:48:47.443882: W tensorflow/compiler/tf2tensorrt/util

In [9]:
# if client.has_collection(collection_name=collection_name):
#     client.drop_collection(collection_name=collection_name)

# client.create_collection(
#     collection_name=collection_name,
#     dimension=embedding_dim,
# )

In [10]:
# data = [
#     {"id": i, "vector": embeddings[i], "text": data_raw[i], "dataset": dataset_path}
#     for i in range(len(embeddings))
# ]
# # print("Data has", len(data), "entities, each with fields: ", data[0].keys())
# # print("Vector dim:", len(data[0]["vector"]))
# data[:2]

In [11]:
# batch_size = 100

# # Batch the data and insert
# for i in range(0, len(data), batch_size):
#     batch_data = data[i:i+batch_size]
    
#     res = client.insert(collection_name=collection_name, data=batch_data)

# print(res)